## Collecting the files with results
Cracked 9.10.
1) Swagger - not used directly:
    - https://is.czechfencing.cz/swagger/v1/swagger.json
    
2) example of proposition file: 
    - https://is.czechfencing.cz/storage/v1/tournament-storage/3423a5a7-f168-453c-8f21-d54c158fe878/***.pdf

3) example of tournamet detail: 
    -https://www.czechfencing.cz/tournament/competition/detail/**7baba914-702c-42b0-a6c6-dddada91c597**

4) "public zone" derived from this ID (also happens in Network when clicked button Results): 
    - https://www.czechfencing.cz/api/public-zone/tournaments/competitions/detail/**7baba914-702c-42b0-a6c6-dddada91c597**

5) content of this public info -> results: 
    - "resultsUrl":"/storage/v1/tournament-storage/**dd4d958e-2383-4a6a-b4b1-d69504c5955f**/XYZ"

6) example of results - see that there is "is.czechfencing.cz": 
    - is.  .. for storages, and for swagger
    - https://is.czechfencing.cz/storage/v1/tournament-storage/**dd4d958e-2383-4a6a-b4b1-d69504c5955f**/ABC.pdf

7) LISTS based on swagger:
    - this is all in  SWAGGER: /boards, /clubs, /tournaments, /section, /competition, /competitions, /contracts, /order, /people, /referees, /person, /region, /requests, /role-sections, /seasons, /sport-fields, 
    - https://www.czechfencing.cz/api/public-zone/tournaments/full - všechny turnaje !!! + Ids ! 2013-2024
    - https://www.czechfencing.cz/api/public-zone/clubs - všechny kluby !
    - https://www.czechfencing.cz/api/public-zone/people - všichni AKTIVNÍ lidé (bez Báry, PK,...) ale s ADRESAMI
    - https://www.czechfencing.cz/api/public-zone/referees - všichni rozhodčí
    - https://www.czechfencing.cz/api/public-zone/region - seznam všech krajů
    - TODO: /api/public-zone/tournaments/competitions/participants

STATS:
- 25 min to call all 3500 competition links to get details of storage
- 50 min to call all 1800 competitions with results and save the PDFs

In [3]:
import pandas as pd
from tqdm import tqdm
import json
import requests
import os
import time


In [14]:
url = f"https://www.czechfencing.cz/api/public-zone/tournaments/full"
response = requests.get(url)
data = response.json()
display(data)

# path = "c:\\Users\\jirka\\Downloads\\fencing_results_all\\lists_of_all\\fencing pokusy 5 - všechny turnaje2.json"
# with open(path, 'r', encoding='utf-8') as file:
#     df = json.load(file)
    
# for key in df:
#     print(key)
    
# print(len(df["data"]))
# df["data"]

{'data': [{'competitions': [{'id': '17918a0b-c0b3-4726-9a9d-ecebf8ec35b7',
     'name': 'Pohár Atmosu - Kord - Mladší žačky',
     'ageCategoryId': 8,
     'sportDisciplineId': 1,
     'previousCompetitionId': None,
     'previousCompetitionName': None,
     'dateFrom': '2021-10-30T00:00:00.000000',
     'dateTill': '2021-10-30T00:00:00.000000'},
    {'id': 'e725ed29-41b0-4944-b6f7-b629dbb23845',
     'name': 'Pohár Atmosu - Kord - Mladší žáci',
     'ageCategoryId': 7,
     'sportDisciplineId': 1,
     'previousCompetitionId': None,
     'previousCompetitionName': None,
     'dateFrom': '2021-10-30T00:00:00.000000',
     'dateTill': '2021-10-30T00:00:00.000000'}],
   'id': '00347b04-cf34-43dd-829f-24893395efb2',
   'name': 'Pohár Atmosu',
   'code': None,
   'dateFrom': '2021-10-30T00:00:00.000000',
   'dateTill': '2021-10-30T00:00:00.000000',
   'seasonId': '5b2a327d-4859-4c20-9dc6-ea785de3b424',
   'organizerId': None,
   'organizerText': None,
   'sportFieldId': None,
   'sportFiel

In [4]:
def get_tournament_links(previous_excel=None):

    # path = "c:\\Users\\jirka\\Downloads\\fencing_results_all\lists_of_all\\fencing pokusy 5 - všechny turnaje.json"

    # with open(path, 'r', encoding='utf-8') as file:
    #     df = json.load(file)

    records = []
    
    url = f"https://www.czechfencing.cz/api/public-zone/tournaments/full"
    response = requests.get(url)
    df = response.json()
    print(f"There are {len(df['data'])} tournaments.")
    
    if previous_excel is not None:
        df_old = pd.read_excel(previous_excel)
        df_old = df_old["competition_id"].unique().tolist()
        print(f"There are {len(df_old)} old tournaments saved.")
    else:
        df_old = []
    

    for rec in tqdm(df["data"]):

        for competition in rec["competitions"]:
            
            #TODO: SKIP the already saved competition - but there might be new RESULTS even for already saved tornaments!!!
            if competition["id"] in df_old:
                continue
            
            record = {}
            record["event_id"] = rec["id"]
            record["competition_id"] = competition["id"]
            record["name"] = competition["name"]
            record["dateFrom"] = competition["dateFrom"]
            
            url = f"https://www.czechfencing.cz/api/public-zone/tournaments/competitions/detail/{competition['id']}"
            response = requests.get(url)
            data = response.json()
            
            try:
                record["storage_url"] = data["resultsUrl"]
            except:
                record["storage_url"] = None 
        
            records.append(record)
            
    res = pd.DataFrame(records)
    
    return res

In [5]:
path =  "c:\\Users\\jirka\\Downloads\\fencing_results_all\\lists_of_all\\all_tournaments.xlsx"
complet_df = get_tournament_links(path)

complet_df

#complet_df.to_excel("c:\\Users\\jirka\\Downloads\\fencing_results_all\lists_of_all\\complet_df_X.xlsx")

There are 1413 tournaments.
There are 3479 old tournaments saved.


100%|██████████| 1413/1413 [00:10<00:00, 135.36it/s]


,event_id,competition_id,name,dateFrom,storage_url
0,22b367f5-83d7-4bf7-9c78-34967fb6276d,ca5613f1-921e-4669-b4c0-3a20c416c210,MT Reutlingen (allstar Cup) – Kord – Seniorky,2024-10-20T09:00:00.000000,None
1,3d8fc790-aed2-44d9-b721-c32f99c831d8,67143d10-3589-48e6-8a11-c44f37c8869e,Velká cena Letňan – Kord – Junioři,2024-11-16T10:00:00.000000,None
2,3d8fc790-aed2-44d9-b721-c32f99c831d8,bd072f9a-c227-40e6-ae93-80182575f038,Velká cena Letňan – Kord – Juniorky,2024-11-16T13:30:00.000000,None
3,606efd9e-1453-4548-8800-d2ebf708ae9c,a5ba42d1-afc4-40e6-8b8b-df56c72c8a4b,O pohár starosty města – Kord – Junioři,2024-12-08T09:00:00.000000,None
4,606efd9e-1453-4548-8800-d2ebf708ae9c,adcdc9cc-42d4-4458-8f5a-823dc470c2e1,O pohár starosty města – Kord – Juniorky,2024-12-08T13:00:00.000000,None
5,606efd9e-1453-4548-8800-d2ebf708ae9c,c22243ae-8c3f-4e24-be24-ae40fe997aae,O pohár starosty města – Kord – Kadetky,2024-12-07T13:20:00.000000,None
6,606efd9e-1453-4548-8800-d2ebf708ae9c,edfb5095-89d6-4975-945e-34d844d1dc84,O pohár starosty města – Kord – Kadeti,2024-12-07T09:00:00.000000,None
7,64ae456e-a9d4-4a27-abed-e124c99b0f2b,2eec8c33-db60-410d-b1f2-95bf27ee9550,Velká cena Dukly Olomouc – Kord – Žačky,2024-11-30T09:30:00.000000,None
8,64ae456e-a9d4-4a27-abed-e124c99b0f2b,423dc5bf-9d9f-4b78-83fa-d3f18cf0c8c5,Velká cena Dukly Olomouc – Kord – Mini žačky,2024-11-30T13:30:00.000000,None
9,64ae456e-a9d4-4a27-abed-e124c99b0f2b,53c52819-ade5-4df6-89fd-903a1d764062,Velká cena Dukly Olomouc – Kord – Žáci,2024-12-01T09:30:00.000000,None


In [7]:
print(len(complet_df["event_id"].unique()))
print(len(complet_df["competition_id"].unique()))

NameError: name 'complet_df' is not defined

In [101]:
for url in tqdm(complet_df["storage_url"].unique()):
    
    if url is not None:
        result_url = f"https://is.czechfencing.cz{url}"
        #print(result_url)
        response = requests.get(result_url)
        
        if response is not None:
            file_path = "c:\\Users\\jirka\\Downloads\\fencing_results_all"
            try:
                tournament_name = url.split("/")[-1].split(".")[0]
                tournament_code = url.split("/")[-2]
                final_name = f"{file_path}\\{tournament_name}__{tournament_code}"
                
                # if os.path.exists(f"{final_name}.pdf"):
                #     copy_count = 1
                #     new_name = f"{final_name}_copy"
                #     while os.path.exists(f"{new_name}.pdf"):
                #         copy_count += 1
                #         new_name = f"{new_name}_copy{copy_count}"
                #     final_name = new_name
                
                with open(f"{final_name}.pdf", 'wb') as file:
                    file.write(response.content)
                    
            except:
                print(f"FAILED TO SAVE: {url}")
        else:
            print(f"FAILED TO OBTAIN: {url}")
    else:
        print(f"there is not available storage URL")

  0%|          | 2/1863 [00:00<13:25,  2.31it/s]

there is not available storage URL


100%|██████████| 1863/1863 [51:08<00:00,  1.65s/it] 
